**PROJECT NOTES**

- Adobe recommends ~150 picture source library for photomosaics, but given that we aren't using any colorization, I think we will probably need more (maybe 300+?)

# The Main Idea

We want to split our input image (or camera frame) into $n$ x $m$ tiles (where $n, m$ are proportional to the aspect ratio of the input), each with a square resolution of $N$ x $N$ pixels. For each tile, we will calculate the "average RGB" value of the pixels within it, which is represented by:

\begin{equation}
(r,g,b)_{avg} = (\frac{r_1 + r_2 + ... + r_{N^2}}{N^2}, \frac{g_1 + g_2 + ... + g_{N^2}}{N^2}, \frac{b_1 + b_2 + ... + b_{N^2}}{N^2})
\end{equation}

(where $(r_1,b_1,g_1), ... , (r_{N^2},b_{N^2},g_{N^2})$ correspond to the RGB values of the $N^2$ pixels in our tile)

It is worth noting that averaging across RGB values is preferable over HSV, HSL, or CMY as we are working with digital images and we want each channel to work equally and independently. For example, the HSL vectors (0, 100, 99) and (180, 0, 99) both correspond to an almost identical white color, despite the distance between these vectors being very large.



---


After computing $(r,g,b)_{avg}$ for a tile, we want to match it to the painting from our image library that has the **closest** average RGB value to the tile (to do this, we would need to compute $(r,g,b)_{avg}$ for all images in our input library). Let: $I = \{\vec{x}^{\,}_{k} = (r,g,b)_{avg,I_k} \mid I_k \text{ the $k$th image in image library}\}$

By "closest", we mean "the distance between $(r,g,b)_{avg}$ and $(r,g,b)_{avg,I_k}$ is minimized". We can do this via the L2-norm:

\begin{equation}
I_{tile} = \arg \min_{I_k} \sqrt{(r_{avg} - r_{avg,I_k})^2 + (g_{avg} - g_{avg,I_k})^2 + (b_{avg} - b_{avg,I_k})^2}
\end{equation}

And there we go! We just need to construct an $n$ x $m$ mosaic image, where each tile $I_{tile}$ corresponds to an image in the library. Resolution downsizing will probably need to be done to each $I_{tile}$ so we don't have a 16384x16384 image lol 

In [ ]:
# Imports

from PIL import Image
import numpy as np
import imghdr
import cv2
import os
from os import listdir
import math
from google.colab import files

In [ ]:
# Mount Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# NOTE: This will require the Painting Library to be cloned into your Google Drive under the same directory name
folder_dir = '/content/drive/MyDrive/Painting Library - Math 155 Final Project'

numjpg = numpng = numwebp = numjpeg = 0

for images in os.listdir(folder_dir):
  if images.endswith('.png'):
    numpng += 1
  elif images.endswith('.jpg'):
    numjpg += 1
  elif images.endswith('.jpeg'):
    numjpeg += 1
  elif images.endswith('.webp'):
    numwebp += 1
  else:
    print("Unrecognized file type detected\n")
    print("\tFile Name: ", images, "\n")

print("Number of .jpg files: ", numjpg, "\n")
print("Number of .png files: ", numpng, "\n")
print("Number of .jpeg files: ", numjpeg, "\n")
print("Number of .webp files: ", numwebp, "\n")


Number of .jpg files:  95 

Number of .png files:  31 

Number of .jpeg files:  39 

Number of .webp files:  8 



In [ ]:
# Crop/Resize (thanks Anna)

def crop_and_resize(im):
  width, height = im.size
  imageSize = 256

  if width >= height:
    diff = width - height

    left = math.floor(diff/2)
    right = math.floor(width - diff/2)
    top = 0
    bottom = height

  else:
    diff = height - width

    left = 0
    right = width
    top = math.floor(diff/2)
    bottom = math.floor(height - diff/2)

  im = im.crop((left, top, right, bottom))

  # resize

  im = im.resize((imageSize, imageSize))

  return im 






In [ ]:
# File Conversion time! (to jpg)

# Make output path (if one does not exist already)

path_dir = '/content/drive/MyDrive/Painting_Library_Converted'
if not os.path.exists(path_dir):
  os.mkdir(path_dir)
  print("Directory Painting_Library_Converted Created")
else:
  for file in os.listdir(path_dir):
    os.remove(path_dir + '/' + file) # BE CAREFUL WITH MODIFYING THIS!!! IT CAN DELETE FILES IN DRIVE
                                     # THIS IS JUST TO ENSURE THAT WE DO NOT DUPLICATE FILES

counter = 0
for images in os.listdir(folder_dir):
  im = Image.open(folder_dir + '/' + images).convert("RGB")

  if images.endswith('.jpg') or images.endswith('.png'):
    images = images[:-4]
  elif images.endswith('.webp') or images.endswith('.jpeg'):
    images = images[:-5]
  else:
    print("Error!")
    break

  im = crop_and_resize(im)
  
  im.save(path_dir + '/' + images + '.jpg', 'JPEG')
  # im.save(path_dir + '/' + str(counter) + '.jpg', 'JPEG')
  # ^^ Counter is for to-js library

  counter += 1



/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (164084688 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (136118889 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
im = Image.open(path_dir + '/' + '640px-Boris_Kustodiev_-_Shrovetide_-_Google_Art_Project.jpg')
array = np.asarray(im)
print(array.shape)
print(len(array))

(256, 256, 3)
256


# Crop and Resize (MATLAB ver.)


```
imageSize = 256; % N x N square, can modify the actual size

directory = '/content/drive/MyDrive/Painting_Library_Converted';

% Get a list of image files in the directory
imageFiles = dir(fullfile(directory, '*.jpg')); 

for i = 1:numel(imageFiles)
    imagePath = fullfile(directory, imageFiles(i).name);
    orig_img = imread(imagePath);

    % Crop the image to a square
    [height, width, ~] = size(orig_img);
    minDim = min(height, width);
    left = (width - minDim) / 2;
    top = (height - minDim) / 2;
    right = left + minDim;
    bottom = top + minDim;
    croppedImage = orig_img((top+1):bottom, (left+1):right,1:3);

    % Resize the image to the desired size
    resizedImage = imresize(croppedImage, [imageSize, imageSize]);

    % Save the modified image
    [~, imageName, imageExt] = fileparts(imageFiles(i).name);
    modifiedImagePath = fullfile(directory, [imageName, '_modified', imageExt]);
    imwrite(resizedImage, modifiedImagePath);

end
```


In [ ]:
# Compute average RGB values for given neighborhood of pixels
def compute_avg_rgb(pixels):
  n = pixels.shape[0] * pixels.shape[1] 
  total_r = total_g = total_b = 0
  for row in pixels:
    for r, g, b in row:
      total_r += r
      total_g += g
      total_b += b

  avg_r = total_r / n
  avg_g = total_g / n
  avg_b = total_b / n

  return (avg_r, avg_g, avg_b)

In [ ]:
# Compute average RGB values for each painting in the converted library
def avg_rgb_paintings(paintings_directory):
  avg_rgb = {}

  for filename in os.listdir(paintings_directory):
    if filename.endswith(".jpg"):
      image_path = os.path.join(paintings_directory, filename)
      image = Image.open(image_path)

      pixel_data = np.asarray(image)

      avg_rgb[filename] = compute_avg_rgb(pixel_data)

  return avg_rgb

In [ ]:
# Find the painting with the closest average RGB values 
def find_closest_painting(target_rgb, paintings_rgb):
    closest = None
    min_dist = float('inf')

    for filename in paintings_rgb:
      avg_rgb = paintings_rgb[filename]
      dist = ((target_rgb[0] - avg_rgb[0]) ** 2 + (target_rgb[1] - avg_rgb[1]) ** 2 +
                  (target_rgb[2] - avg_rgb[2]) ** 2) ** 0.5

      if dist < min_dist:
        min_dist = dist
        closest = filename

    return closest


In [ ]:
# Divide the image into tiles and substitute it with a painting with closest average RGB value
def divide_substitute(input_path, output_path, paintings_directory, tile_min, overlap):
    image = Image.open(input_path)

    r_up = 1 # resolution increase parameter

    m, n = image.size

    tile_size = min(m // tile_min, n // tile_min)

    horizontal = (m - overlap) // (tile_size - overlap)
    vertical = (n - overlap) // (tile_size - overlap)

    output = Image.new('RGB', (r_up*m, r_up*n))

    paintings_dict = avg_rgb_paintings(paintings_directory)

    for i in range(vertical):
      for j in range(horizontal):
        x = j * (tile_size - overlap)
        y = i * (tile_size - overlap)
        tile = image.crop((x, y, x + tile_size, y + tile_size))

        tilearray = np.asarray(tile)

        avg_rgb = compute_avg_rgb(tilearray)

        filename = find_closest_painting(avg_rgb, paintings_dict)
        path = os.path.join(paintings_directory, filename)

        painting = Image.open(path)
        resized_painting = painting.resize((r_up*tile_size, r_up*tile_size))

        output.paste(resized_painting, (r_up*x, r_up*y))

    output.save(output_path)

    # output.show()

In [ ]:
# put it altogether

output_path = '/content/drive/MyDrive/Output_Video_Folder'
input_path = '/content/drive/MyDrive/155_final_video'

#k = 0

for image in os.listdir(input_path):
  im = input_path + '/' + image

  input_direction = input_path + '/' + image
  
  output_direction = output_path + '/' + image

  divide_substitute(input_direction, output_direction, path_dir, 60, 0)
  #k += 1


# divide_substitute('/content/drive/MyDrive/japanpicture.jpg', 'b', path_dir, 52, 0) # replace last extension with whatever img u wanna use
# requires jpg